**Умова**

Проведіть класифікацію на основі датасету UCI ML Breat Cancer Wisconsin, використовуючи вказані у презентації модулі бустингу, буггінгу та стекінгу. Змініть їх параметри наступним чином:

- у StackingClassifier як базовий алгоритм встановіть LogisticRegression та DecisionTreeClassifier, як фінальну модель - SVC;
- у BaggingClassifier як базовий алгоритм встановіть DecisionTreeClassifier;
- у AdaBoostClassifier як базовий алгоритм встановіть LogisticRegression;
- y XGBClassifier як базовий алгоритм (booster) встановіть 'gbtree'.

У зведеній таблиці порівняйте результати класифікацій, наведених у коді, із класифікаціями, що отримані вами.

**1. Отримаємо дані датасету та розіб'ємо на тестовий та тренувальний датасети.**

In [1]:
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

data, target = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=.3, random_state=12)

**2. Проведемо класифікацію на основі StackingClassifier.**

In [2]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC

#створення базової моделі
estimators = [('lgr', LogisticRegression(solver='lbfgs', max_iter=3000)),('tree', DecisionTreeClassifier(max_depth=5))]

#створення моделі стекінга
stacking_clf = StackingClassifier(estimators=estimators, final_estimator=SVC(random_state=42))

#навчаємо модель
stacking_clf = stacking_clf.fit(X_train, y_train)

#отримуємо класифікацію з тестового датасуту
y_pred = stacking_clf.predict(X_test)

print("Якість моделі на тестовому наборі: {:.3f}".format(stacking_clf.score(X_test, y_test)))

Якість моделі на тестовому наборі: 0.936


**3. Проведемо класифікацію на основі BaggingClassifier.**

In [3]:
from sklearn.ensemble import BaggingClassifier

modelClf_l = BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=5), n_estimators=15, random_state=20)
modelClf_l = modelClf_l.fit(X_train, y_train)

print("Якість моделі на тестовому наборі: {:.3f}".format(modelClf_l.score(X_test, y_test)))

Якість моделі на тестовому наборі: 0.918


**4. Проведемо класифікацію на основі AdaBoostClassifier.**

In [4]:
from sklearn.ensemble import AdaBoostClassifier

modelClf_2 = AdaBoostClassifier(estimator=LogisticRegression(solver='lbfgs', max_iter=3000), 
                                n_estimators=100, random_state=12)
modelClf_2 = modelClf_2.fit(X_train, y_train)
print("Якість моделі на тестовому наборі: {:.3f}".format(modelClf_2.score(X_test, y_test)))

Якість моделі на тестовому наборі: 0.936


**5. Проведемо класифікацію на основі XGBClassifier.**

In [6]:
from xgboost import XGBClassifier

modelClf_3 = XGBClassifier(booster='gbtree', learning_rate=0.02, n_estimators=10)
modelClf_3 = modelClf_3.fit(X_train, y_train)

print("Якість моделі на тестовому наборі: {:.3f}".format(modelClf_3.score(X_test, y_test)))

Якість моделі на тестовому наборі: 0.912


**6. Побудуємо зведену таблиці для порівняння результатів.**

In [8]:
import pandas as pd

df = pd.DataFrame()
df["work"] = ["Example", "New_analysis"]
df["StackingClassifier"] = [0.912, round(stacking_clf.score(X_test, y_test), 3)]
df["BaggingClassifier"] = [0.941, round(modelClf_l.score(X_test, y_test), 3)]
df["AdaBoostClassifier"] = [0.906, round(modelClf_2.score(X_test, y_test), 3)]
df["XGBClassifier"] = [0.854, round(modelClf_3.score(X_test, y_test), 3)]
df["mean"] = round(df.iloc[:, 1:].mean(axis=1), 3)
df["max"] = df.iloc[:, 1:-1].max(axis=1)
df["min"] = df.iloc[:, 1:-1].min(axis=1)
df

,work,StackingClassifier,BaggingClassifier,AdaBoostClassifier,XGBClassifier,mean,max,min
0,Example,0.912,0.941,0.906,0.854,0.903,0.941,0.854
1,New_analysis,0.936,0.918,0.936,0.912,0.926,0.936,0.912
